In [1]:
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:

#Imports
from io import BytesIO
import time
import csv
import os
import open3d as o3d
import random
import requests
import tarfile
import numpy as np
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
#%%
def gromov_wasserstein(pc1: np.ndarray, pc2: np.ndarray) -> float:
    def dist_ecc_fast(ecc, u):
        return (np.mean(ecc <= u))

    out = 0
    # convert point clouds to numpy arrays
    pc1 = np.asarray(pc1.points)
    pc2 = np.asarray(pc2.points)

    # Reshape input matrices if necessary
    if pc1.ndim == 1:
        pc1 = pc1.reshape(-1, 1)
    if pc2.ndim == 1:
        pc2 = pc2.reshape(-1, 1)

    ecc1 = squareform(pdist(pc1)).mean(0)
    ecc2 = squareform(pdist(pc2)).mean(0)
    unique_ecc = np.unique(np.concatenate((ecc1, ecc2)))
    for i in range(unique_ecc.shape[0] - 1):
        u = unique_ecc[i]
        out += (unique_ecc[i + 1] - unique_ecc[i]) * np.abs(dist_ecc_fast(ecc1, u) - dist_ecc_fast(ecc2, u))

    return (0.5 * out)
#%%
def create_pcd_from_mesh(mesh):
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh])
    # distribute dots evenly on the surface
    return mesh.sample_points_uniformly(500)
#%%
def load_model(link, path):
    # http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/
    response = requests.get(link)
    tgz_data = BytesIO(response.content)
    # set the current working directory to the script's directory
    script_directory = os.path.dirname(os.path.abspath(__file__))
    os.chdir(script_directory)
    with tarfile.open(fileobj=tgz_data, mode="r:gz") as tar_ref:
        tar_ref.extractall(script_directory)
    # join paths
    model_path = os.path.join(script_directory, path, "clouds", "merged_cloud.ply")
    # load pointcloud
    pcd = o3d.io.read_point_cloud(model_path)
    return pcd
#%%
def load_cad_model(model):
    # load model generated in freecad
    return o3d.io.read_point_cloud(model)
#%%
def visualize_model(model):
    o3d.visualization.draw_geometries([model])
#%%
def get_num_points(model):
    print(len(model.points))
#%%
def create_pointcloud_from_coordinates(coordinates):
    # create point cloud with coordinates
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coordinates)
    return pcd
#%%
def get_coordinates(model):
    coordinates = [list(point) for point in model.points]
    # print(coordinates[:50])
    return coordinates
#%%
def random_downsampling(model, endpoints):
    # get coordinates of the models
    coordinates = get_coordinates(model)
    # select random points for downsampling
    for i in range(len(coordinates) - endpoints):
        rannumb = random.randint(0, len(coordinates) - 1)
        del coordinates[rannumb]
    point_cloud = create_pointcloud_from_coordinates(coordinates)
    return point_cloud
#%%
def farthest_point_sampling(model, num_points_keep):
    coordinates = np.array(get_coordinates(model))
    retVal = []
    # to make runs comparable
    random.seed(13)
    # generate "random" int
    randint = random.randint(0, len(coordinates) - 1)
    # select random point from model
    retVal.append(coordinates[randint])
    # delete chosen point from original model after it was added to the downsampled cloud
    coordinates = np.delete(coordinates, randint, axis=0)
    while len(retVal) < num_points_keep:
        # Berechne die euklidischen Distanzen der ausgewählten Punkte zu den verbleibenden Punkten
        eucl_distances = distance.cdist(retVal, coordinates, 'euclidean')
        # Find point with largest min Distance
        min_mindist = np.min(eucl_distances, axis=0)
        # Find index of point with largest min DIstance
        max_min_distance_index = np.argmax(min_mindist)
        # add point that is farthest away
        retVal.append(coordinates[max_min_distance_index])
        # delete point from coordinates list
        coordinates = np.delete(coordinates, max_min_distance_index, axis=0)
    return create_pointcloud_from_coordinates(np.array(retVal))
#%%
# built in function von open3d
def radius_outlier_removal_call(model):
    return model.remove_radius_outlier(nb_points=5, radius=0.05)
#%%
# add noise to pointcloud
def add_noise(model, noise_value):
    points = np.asarray(model.points)
    noise = np.random.normal(0, noise_value, size=points.shape)
    noisy_points = points + noise

    noisy_pc = o3d.geometry.PointCloud()
    noisy_pc.points = o3d.utility.Vector3dVector(noisy_points)
    return noisy_pc
#%%
def create_voxel_grid(model, voxel_size):
    model_points = np.array(get_coordinates(model))
    min_bound = np.min(model_points, axis=0)
    max_bound = np.max(model_points, axis=0)

    dimensions = np.ceil((max_bound - min_bound) / voxel_size).astype(int)

    voxelgrid = np.zeros(dimensions)

    for point in model_points:
        voxel_coordinates = ((point - min_bound) / voxel_size).astype(int)
        # -1 needed in order to avoid index out of bounds
        voxelgrid[tuple(voxel_coordinates - 1)] += 1
    # convert voxelgrid to open3d Voxelgrid
    o3d_voxelgrid = o3d.geometry.VoxelGrid.create_from_point_cloud(input=model, voxel_size=voxel_size)
    #o3d.visualization.draw_geometries([o3d_voxelgrid])
    return o3d_voxelgrid


def voxel_filter(model, voxelgrid, voxel_size):
    # list where downsampled points will be saved
    downsampled_points = []
    # iterate over all voxel in the voxelgrid
    for voxel in voxelgrid.get_voxels():
        # get bounds of the voxel
        downsampled_points.extend(is_point_in_voxel(model, voxelgrid, voxel, voxel_size))
    downsampled_points = np.asarray(downsampled_points)
    return create_pointcloud_from_coordinates(downsampled_points)


def aggregate_points(points):
    # Aggregate the points by averaging, taking into account the z coordinate
    if len(points) == 0:
        return points
    aggregated_points = []
    aggregated_points.append(np.mean(points, axis=0))
    return aggregated_points


def is_point_in_voxel(model, voxelgrid, voxel, voxel_size):
    # get center point and see whether a point lies within the given distance/2 of the voxel size from the center
    voxel_center = voxelgrid.get_voxel_center_coordinate(voxel.grid_index)
    points_in_voxel = []
    half_size = voxel_size / 2.0
    # check, which points are lying within a voxel
    for point in model.points:
        if np.all(np.abs(point - voxel_center) <= half_size):
            points_in_voxel.append(point)
    points_in_voxel = aggregate_points(points_in_voxel)
    # print(points_in_voxel)
    return points_in_voxel


def create_points_from_voxel(voxel_model):
    # convert vector in numpy array
    vector_array = np.asarray(voxel_model)

    # create o3d point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(vector_array)

    return point_cloud

#%%
def point_cloud_to_ply(point_cloud, file_name):
    # safe downsampled point cloud as ply data
    #file_name = "point_cloud_images/" + file_name + ".ply"
    file_name = file_name + ".ply"
    #if os.path.exists("point_cloud_images/" + file_name):
    #    os.remove(file_name)
    if os.path.exists(file_name):
        os.remove(file_name)
    o3d.io.write_point_cloud(file_name, o3d.geometry.PointCloud(point_cloud.points))

#%%
cone = load_cad_model(r"/cone.ply")
sphere = load_cad_model(r"/sphere.ply")
cube = load_cad_model(r"/cube.ply")
complex_cube = load_cad_model(r"/complexCube.ply")
complex_cone = load_cad_model(r"/hollowCone.ply")
complex_sphere = load_cad_model(r"/complexSphere.ply")
pencil = load_cad_model(r"/pencil_fein.ply")
# source: https://sketchfab.com/3d-models/davis-teapot-materialcleanup-547971eaf21d43f2b6cfcb6be0e7bf11
teapot = load_cad_model(r"/teapot.ply")
# source: https://sketchfab.com/3d-models/book-ba04f5ac66194341bc7d437fb6c94674
book = load_cad_model(r"/book.ply")
#%%
def icp_algorithm(source, target):
    # transform target point cloud
    transformation = np.array([[0.86, 0.5, 0.1, 0.5],
                               [-0.5, 0.86, 0.1, 0.99],
                               [0.0, -0.1, 0.99, 0.5],
                               [1.3, 0.0, 0.0, 1.0]])
    target = target.transform(transformation)

    threshold = 0.25  # max distance for deleting points
    initial_transformation = np.identity(4)  # initial guess of transformation

    # Open3D ICP Algorithmus
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, initial_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    source.transform(reg_p2p.transformation)
    return reg_p2p

#%%
def write_csv(array, filename):
    # Öffne die CSV-Datei im Schreibmodus
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        for row in array:
            writer.writerow([row])
#%%

In [10]:

rd_wasserstein = []
vf_wasserstein = []
fp_wasserstein = []

num_iterations = 25
model_array = [cone]
array_noise = [0.05, 0.2, 1]
for model in model_array:
    for noise in array_noise:
        for i in range(num_iterations):
            model = add_noise(model, noise)

            #random downsampling
            rd = random_downsampling(model, int(len(model.points) / 10 * 0.2))

            # voxelgrid
            vx_grid = create_voxel_grid(model, 8.5)
            vx = voxel_filter(model, vx_grid, 8.5)

            # farthest point downsampling
            fp = farthest_point_sampling(model, int(len(model.points) / 10 * 0.2))

            # Random Downsampling
            rd_wasserstein.append(gromov_wasserstein(rd, model))
            # Voxelgrid Filter
            vf_wasserstein.append(gromov_wasserstein(vx, model))
            # Farthest Point Downsampling
            fp_wasserstein.append(gromov_wasserstein(fp, model))

write_csv(rd_wasserstein, "rd_wasserstein.csv")
write_csv(vf_wasserstein, "vf_wasserstein.csv")
write_csv(fp_wasserstein, "fp_wasserstein.csv")

#%%


In [3]:
rd_times = []  # List of lists for random downsampling times
vf_times = []  # List of lists for voxel filter times
fp_times = []  # List of lists for farthest point sampling times

num_iterations = 10

# Methode Spalte, Modell Zeile ?
# [][][]
# [][][]
model_array = [cone]
for i, model in enumerate(model_array):
    for round in range(num_iterations):
        # random downsampling
        start = time.time()
        rd = random_downsampling(model, int(len(model.points) / 10 * 0.2))
        end = time.time()
        elapsed_time = end - start
        rd_times.append(elapsed_time)

        # voxelgrid filter
        start = time.time()
        vx_grid = create_voxel_grid(model, 8.5)
        vx = voxel_filter(model, vx_grid, 8.5)
        end = time.time()
        elapsed_time = end - start
        vf_times.append(elapsed_time)

        # farthest point downsampling
        start = time.time()
        fp = farthest_point_sampling(model, int(len(model.points) / 10 * 0.2))
        end = time.time()
        fp_pc = o3d.geometry.PointCloud()
        fp_pc.points = o3d.utility.Vector3dVector(np.asarray(fp.points))
        elapsed_time = end - start
        fp_times.append(elapsed_time)  # Add the time to the corresponding model's list

write_csv(rd_times, "rd_times.csv")
write_csv(vf_times, "vf_times.csv")
write_csv(fp_times, "fp_times.csv")
#%%


In [9]:
# Vergleich auf den Originalwolken
original_fitness = []
original_inlier = []
original_icp_time = []


model_array = [cone]

for model in model_array:
    # ICP auf den Originalpunktewolken
    start = time.time()
    org_icp = icp_algorithm(model, model)
    end = time.time()
    original_icp_time.append(end - start)
    original_fitness.append(org_icp.fitness)
    original_inlier.append(org_icp.inlier_rmse)

write_csv(original_inlier, "original_icp_inlier.csv")
write_csv(original_fitness, "original_icp_fitness.csv")
write_csv(original_icp_time, "original_icp_time.csv")

#%%


In [8]:
rd_icp_fitness = []
rd_icp_inlier = []
vf_icp_fitness = []
vf_icp_inlier = []
fp_icp_fitness = []
fp_icp_inlier = []
fp_icp_time = []
vf_icp_time = []
rd_icp_time = []



model_array = [cone]


for model in model_array:
    # random downsampling
    model_rd = random_downsampling(model, int(len(model.points) / 10 * 0.2))
    model_rd_pc = o3d.geometry.PointCloud()
    model_rd_pc.points = o3d.utility.Vector3dVector(np.asarray(model_rd.points))
    start = time.time()
    rd_icp = icp_algorithm(model_rd_pc, model)
    end = time.time()
    rd_icp_fitness.append(rd_icp.fitness)
    rd_icp_inlier.append(rd_icp.inlier_rmse)
    rd_icp_time.append(end - start)
    #model_transformed = model.transform(rd_icp.transformation)

    # show source and source and target
    #o3d.visualization.draw_geometries([model, model_transformed])

    # voxel grid filter
    model_voxel_grid = create_voxel_grid(model, 8.5)
    model_voxel = voxel_filter(model, model_voxel_grid, 8.5)
    start = time.time()
    vf_icp = icp_algorithm(model_voxel, model)
    end = time.time()
    vf_icp_time.append(end - start)
    vf_icp_fitness.append(vf_icp.fitness)
    vf_icp_inlier.append(vf_icp.inlier_rmse)
    #model_transformed = model.transform(vf_icp.transformation)

    # show source and source and target
    #o3d.visualization.draw_geometries([model, model_transformed])

    # farthest point downsampling#
    model_fp = farthest_point_sampling(model, int(len(model.points) / 10 * 0.2))
    model_fp_pc = o3d.geometry.PointCloud()
    model_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(model_fp.points))
    start = time.time()
    fp_icp = icp_algorithm(model_fp_pc, model)
    end = time.time()
    fp_icp_time.append(end - start)
    fp_icp_fitness.append(fp_icp.fitness)
    fp_icp_inlier.append(fp_icp.inlier_rmse)
    #model_transformed = model.transform(fp_icp.transformation)

    # show source and source and target
    #o3d.visualization.draw_geometries([model, model_transformed])

write_csv(rd_icp_fitness, "rd_icp_fitness.csv")
write_csv(vf_icp_fitness, "vf_icp_fitness.csv")
write_csv(fp_icp_fitness, "fp_icp_fitness.csv")
write_csv(rd_icp_inlier, "rd_icp_inlier.csv")
write_csv(vf_icp_inlier, "vf_icp_inlier.csv")
write_csv(fp_icp_inlier, "fp_icp_inlier.csv")
write_csv(rd_icp_time, "rd_icp_time.csv")
write_csv(vf_icp_time, "vf_icp_time.csv")
write_csv(fp_icp_time, "fp_icp_time.csv")
#%%


In [4]:
model_array = [cube, cone, sphere]
model_names = ["cube", "cone", "sphere"]
for index, model in enumerate(model_array):
    # create noisy pointclouds
    noisy_model = add_noise(model, 1.3)

    # Random Downsampling
    rd_noisy = random_downsampling(noisy_model, int(len(noisy_model.points) / 10 * 0.2))
    point_cloud_to_ply(rd_noisy, "noisy_rd_" + model_names[index - 1])

    # Voxel Grid Filter
    noisy_model_grid = create_voxel_grid(noisy_model, 8.5)
    noisy_model_voxel_pc = voxel_filter(noisy_model, noisy_model_grid, 8.5)
    point_cloud_to_ply(noisy_model_voxel_pc, "noisy_vf_" + model_names[index - 1])

    # Farthest Point Downsampling
    noisy_model_fp = farthest_point_sampling(noisy_model, int(len(noisy_model.points) / 10 * 0.2))
    noisy_model_fp_pc = o3d.geometry.PointCloud()
    noisy_model_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(noisy_model_fp.points))
    point_cloud_to_ply(noisy_model_fp_pc, "noisy_fp_" + model_names[index - 1])

#%%
model_array = [complex_sphere, complex_cube, complex_cone]
model_names = ["complex_sphere", "complex_cube", "complex_cone"]

for index, complex_model in enumerate(model_array):
    noisy_complex_model = add_noise(complex_model, 1.3)
    rd_complex_noisy = random_downsampling(noisy_complex_model, int(len(noisy_complex_model.points) / 10 * 0.2))
    point_cloud_to_ply(rd_complex_noisy, "noisy_rd_" + model_names[index])

    noisy_complex_model_grid = create_voxel_grid(noisy_complex_model, 8.5)
    noisy_complex_model_voxel_pc = voxel_filter(noisy_complex_model, noisy_complex_model_grid, 8.5)
    point_cloud_to_ply(noisy_complex_model_voxel_pc, "noisy_vf_" + model_names[index])

    noisy_complex_model_fp = farthest_point_sampling(noisy_complex_model, int(len(noisy_complex_model.points) / 10 * 0.2))
    noisy_complex_model_fp_pc = o3d.geometry.PointCloud()
    noisy_complex_model_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(noisy_complex_model_fp.points))
    point_cloud_to_ply(noisy_complex_model_fp_pc, "noisy_fp_" + model_names[index])
#%%



In [ ]:
model_array = [book, teapot, pencil]
model_names = ["book", "teapot", "pencil"]

for index, model_object in enumerate(model_array):
    # random downsampling
    noisy_model_object = add_noise(model_object, 1.3)
    rd_noisy = random_downsampling(noisy_model_object, int(len(noisy_model_object.points) / 10 * 0.2))
    point_cloud_to_ply(rd_noisy, "noisy_rd_" + model_names[index])

    # voxel grid filter
    noisy_model_object_grid = create_voxel_grid(noisy_model_object, 8.5)
    noisy_object_voxel_pc = voxel_filter(noisy_model_object, noisy_model_object_grid, 8.5)
    point_cloud_to_ply(noisy_object_voxel_pc, "noisy_vf_" + model_names[index])

    # farthest point downsampling
    noisy_model_object_fp = farthest_point_sampling(noisy_model_object, int(len(noisy_model_object.points) / 10 * 0.2))
    noisy_model_object_fp_pc = o3d.geometry.PointCloud()
    noisy_model_object_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(noisy_model_object_fp.points))
    point_cloud_to_ply(noisy_model_object_fp_pc, "noisy_fp_" + model_names[index])
#%%


In [7]:
#buch wieder hinzufügen!
model_array = [cube, cone, sphere, complex_sphere, complex_cube, complex_cone, teapot, pencil]
model_names = ["cube", "cone", "sphere", "complex_sphere", "complex_cube", "complex_cone", "teapot", "pencil"]

for index, model in enumerate(model_array):

    # Random Downsampling
    rd = random_downsampling(model, int(len(model.points) / 10 * 0.2))
    point_cloud_to_ply(rd, "rd_" + model_names[index - 1])

    # Voxel Grid Filter
    model_grid = create_voxel_grid(model, 8.5)
    model_voxel_pc = voxel_filter(model, model_grid, 8.5)
    point_cloud_to_ply(model_voxel_pc, "vf_" + model_names[index - 1])

    # Farthest Point Downsampling
    model_fp = farthest_point_sampling(model, int(len(model.points) / 10 * 0.2))
    model_fp_pc = o3d.geometry.PointCloud()
    model_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(model_fp.points))
    point_cloud_to_ply(model_fp_pc, "fp_" + model_names[index - 1])

#%%

In [ ]:
model_array = [book]
model_names = ["book"]

for index, model in enumerate(model_array):

    # Farthest Point Downsampling
    model_fp = farthest_point_sampling(model, int(len(model.points) / 10 * 0.2))
    model_fp_pc = o3d.geometry.PointCloud()
    model_fp_pc.points = o3d.utility.Vector3dVector(np.asarray(model_fp.points))
    point_cloud_to_ply(model_fp_pc, "fp_" + model_names[index - 1])

#%%

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/my_file.txt', 'w') as f:
  f.write('Hello Google Drive!')

Plotten der Laufzeit Tests

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data1 = pd.read_csv(r"fp_times.csv")
data2 = pd.read_csv(r"vf_times.csv")
data3 = pd.read_csv(r"rd_times.csv")

data_list = [data1['Time'], data2['Time'], data3['Time']]
labels = ['Random Downsampling', 'Voxel Grid Filter', 'Farthest Point Downsampling']

# Erstellen des Boxplots
plt.boxplot(data_list, patch_artist=True, showmeans=True, labels=labels)

# Plot anpassen
plt.title('Boxplot for times of downsampling methods')
plt.xlabel('methods')
plt.ylabel('times')
plt.grid(True)  # Gitter anzeigen (optional)

# Plot anzeigen
plt.show()